In [5]:
from json import load
from os import listdir

test_list = []

for name in listdir("tests"):
    if name.startswith("speedtest") and name.endswith(".json"):
        test_list.append(load(open("tests/" + name)))

print(test_list[0])

[{'start': 'w', 'end': 's', 'time': 0.10049207000000049}, {'start': 'g', 'end': 'b', 'time': 0.09968557899999908}, {'start': 'g', 'end': 'r', 'time': 0.10016319200000012}, {'start': 'z', 'end': 'q', 'time': 0.06650466099999974}, {'start': 'k', 'end': ',', 'time': 0.09954895600000135}, {'start': 'a', 'end': 'z', 'time': 0.11674980200000107}, {'start': 'g', 'end': 'v', 'time': 0.1162943950000006}, {'start': 'o', 'end': '.', 'time': 0.18228782799999976}, {'start': 's', 'end': 'w', 'time': 0.0667862200000009}, {'start': 't', 'end': 'r', 'time': 0.3007095799999995}, {'start': 's', 'end': 'w', 'time': 0.06699562399999692}, {'start': 'z', 'end': 'a', 'time': 0.11685902299999995}, {'start': 'm', 'end': 'h', 'time': 0.13243974299999905}, {'start': 'c', 'end': 'd', 'time': 0.06750744900000072}, {'start': '/', 'end': ';', 'time': 0.2999911750000024}, {'start': 'y', 'end': 'm', 'time': 0.2494832849999966}, {'start': 'x', 'end': 'w', 'time': 0.15033053800000573}, {'start': 'z', 'end': 'a', 'time': 